## Training The Model

In [ ]:
# !pip install transformers
from transformers import DistilBertTokenizer
from transformers import TFDistilBertForSequenceClassification
import tensorflow as tf
import pandas as pd
import json
import gc
df = pd.read_excel("Relevance.xlsx")
# df = pd.read_csv('eda_relevance.csv') # Change path to your download location
df.rename(columns={"Relavance":"target"}, inplace=True)
df['target'].replace(["R","I"],[1,0], inplace =True)
df.head()

,1,target,Data
0,10500,1,__TiffanyAndCo Cousinnnn the return coming soon
1,3597,0,at the balenciaga thinking about my friends fa...
2,10044,1,TiffanyAndCo bracelet I bought in Milan in Oct...
3,4712,0,QueenMoniB personifier channel_gibbs eccentric...
4,2925,0,Pup_Dior_ Happy Valentines Day You are so gor...


In [ ]:
data_texts = df["Data"].to_list() # Features (not-tokenized yet)
data_labels = df["target"].to_list() # Lables

In [ ]:
from sklearn.model_selection import train_test_split

# Split Train and Validation data
train_texts, test_texts, train_labels, test_labels = train_test_split(data_texts, data_labels, test_size=0.3, random_state=0)

In [ ]:
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
train_encodings = tokenizer(train_texts, truncation=True, padding=True)
test_encodings = tokenizer(test_texts, truncation=True, padding=True)

In [ ]:
with tf.device('/GPU:0'):
  train_dataset = tf.data.Dataset.from_tensor_slices((
      dict(train_encodings),
      train_labels
  ))
  test_dataset = tf.data.Dataset.from_tensor_slices((
      dict(test_encodings),
      test_labels
  ))

In [ ]:
model = TFDistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=3)

optimizer = tf.keras.optimizers.legacy.Adam(learning_rate=5e-5)
model.compile(optimizer=optimizer, loss=model.hf_compute_loss, metrics=['accuracy'])

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertForSequenceClassification: ['vocab_projector', 'activation_13', 'vocab_transform', 'vocab_layer_norm']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier', 'pre_classifier', 'dropout_39']
You should probably TRAIN this model on a down-stream task to be able to use i

In [ ]:
with tf.device('/GPU:0'):
  model.fit(train_dataset.shuffle(1000).batch(16), epochs=3, batch_size=16,validation_data=test_dataset.shuffle(1000).batch(16))

Epoch 1/3
189/189 [==============================] - 215s 1s/step - loss: 0.5926 - accuracy: 0.6977 - val_loss: 0.4900 - val_accuracy: 0.7597
Epoch 2/3
189/189 [==============================] - 188s 993ms/step - loss: 0.4133 - accuracy: 0.8236 - val_loss: 0.4848 - val_accuracy: 0.7605
Epoch 3/3
189/189 [==============================] - 187s 992ms/step - loss: 0.2471 - accuracy: 0.9066 - val_loss: 0.7454 - val_accuracy: 0.7395


In [ ]:
save_directory = "/content/drive/MyDrive/CZ4024 Information Retrieval/normalData1_Relevance" # change this to your preferred location

model.save_pretrained(save_directory)
tokenizer.save_pretrained(save_directory)

('/content/drive/MyDrive/CZ4024 Information Retrieval/normalData1_Relevance/tokenizer_config.json',
 '/content/drive/MyDrive/CZ4024 Information Retrieval/normalData1_Relevance/special_tokens_map.json',
 '/content/drive/MyDrive/CZ4024 Information Retrieval/normalData1_Relevance/vocab.txt',
 '/content/drive/MyDrive/CZ4024 Information Retrieval/normalData1_Relevance/added_tokens.json')

In [ ]:
# save_directory = "/content/drive/MyDrive/CZ4024 Information Retrieval/normalData_Polarity"
loaded_tokenizer = DistilBertTokenizer.from_pretrained(save_directory)
loaded_model = TFDistilBertForSequenceClassification.from_pretrained(save_directory, output_attentions=True)

Some layers from the model checkpoint at /content/drive/MyDrive/CZ4024 Information Retrieval/normalData1_Relevance were not used when initializing TFDistilBertForSequenceClassification: ['dropout_39']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at /content/drive/MyDrive/CZ4024 Information Retrieval/normalData1_Relevance and are newly initialized: ['dropout_59']
You should probably TRAIN this model on a down-stream task to be able t

In [ ]:
# !pip install transformers

from sklearn.model_selection import train_test_split
from transformers import DistilBertTokenizer
import tensorflow as tf
import pandas as pd


df = pd.read_csv('eda_polarity.csv') # Change path to your download location
# df = pd.read_excel("Relevance.xlsx")
# print(df.head())
# For Relevance Data
# df.rename(columns={"Relevance":"target"}, inplace=True)
# df['target'].replace(["R","I"],[1,0], inplace =True)

# For Polarity Data
df.rename(columns={"Polarity":"target"}, inplace=True)
df['target'].replace([0,2,4],[0,1,2], inplace =True)
df = df[["target", "Data"]]
# df['target'].replace([-1,0,1],[0,1,2], inplace =True)

print(df.head())

data_texts = df["Data"].to_list() # Features (not-tokenized yet)
data_labels = df["target"].to_list() # Lables

# Split Train and Validation data
train_texts, test_texts, train_labels, test_labels = train_test_split(data_texts, data_labels, test_size=0.3, random_state=7)

tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
train_encodings = tokenizer(train_texts, truncation=True, padding=True)
test_encodings = tokenizer(test_texts, truncation=True, padding=True)

   target                                               Data
0       0    balenciaga and the rise of trash by umair haque
1       0               removed the balenciaga scandal broke
2       0  omega aqua terra i really wanted an explorer p...
3       0  they seem to have a life of their own crazy di...
4       0  since i dont twitter and shoe only put up a mi...


In [ ]:
# !pip install transformers
from transformers import DistilBertTokenizer
from transformers import DistilBertForSequenceClassification
import numpy as np
from sklearn.metrics import classification_report
import torch
save_directory = "/content/drive/MyDrive/CZ4024 Information Retrieval/normalData1_Relevance"
loaded_tokenizer = DistilBertTokenizer.from_pretrained(save_directory)
loaded_model = DistilBertForSequenceClassification.from_pretrained(save_directory, from_tf=True)

All TF 2.0 model weights were used when initializing DistilBertForSequenceClassification.

All the weights of DistilBertForSequenceClassification were initialized from the TF 2.0 model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use DistilBertForSequenceClassification for predictions without further training.


In [ ]:
pred_values = []
for i,test_text in enumerate(test_texts):
  predict_input = loaded_tokenizer.encode(test_text,truncation=True,padding=True,return_tensors="pt")
  output = loaded_model(predict_input)[0]
  prediction_value = torch.argmax(output[0], dim=-1).item()
  pred_values.append(prediction_value)
  if(i%50==0):
    print(i,"/",len(test_texts))
# print(classification_report(test_labels, prediction_value))

0 / 1290
50 / 1290
100 / 1290
150 / 1290
200 / 1290
250 / 1290
300 / 1290
350 / 1290
400 / 1290
450 / 1290
500 / 1290
550 / 1290
600 / 1290
650 / 1290
700 / 1290
750 / 1290
800 / 1290
850 / 1290
900 / 1290
950 / 1290
1000 / 1290
1050 / 1290
1100 / 1290
1150 / 1290
1200 / 1290
1250 / 1290


In [ ]:
# print(classification_report(test_labels, pred_values, target_names=["Neg","Neut","Pos"]))
print(classification_report(test_labels, pred_values, target_names=["Irr","Rel"]))

              precision    recall  f1-score   support

         Irr       0.74      0.88      0.81       792
         Rel       0.73      0.52      0.60       498

    accuracy                           0.74      1290
   macro avg       0.74      0.70      0.71      1290
weighted avg       0.74      0.74      0.73      1290



In [ ]:
for i,label in enumerate(test_labels):
  if label!=1 and label!=0:
    test_labels[i] = 0

In [ ]:
# df[df['target'] == 0]
len(test_labels)

4614

In [ ]:
df = pd.read_csv('eda_relevance.csv')
df['Relevance'].replace(["R","I"],[1,0], inplace =True)
df.rename(columns = {"Relevance":"target"}, inplace = True)
df

,Data,target
0,ksorbs pdaug by now we should have profile pic...,0
1,ksorbs pdaug by now we should have profile pic...,0
2,ksorbs pdaug by now we should profile with spe...,0
3,ksorbs pdaug by now we should have profile pic...,0
4,ksorbs pdaug by now we should have profile pic...,0
...,...,...
34161,hermes would like you to know that alternate s...,1
34162,what this at rolex precision is fake help me pls,1
34163,this is a for something please dont kill me lo...,1
34164,balenciaga handbag logo brown no,1


## Predicting All Data

In [ ]:
!pip install transformers
from sklearn.model_selection import train_test_split
from transformers import DistilBertTokenizer
import tensorflow as tf
import pandas as pd

# twitter_df = pd.read_csv('Cleaned_Tweets.csv') # Change path to your download location
reddit_df = pd.read_csv('Cleaned_Reddit.csv') # Change path to your download location
# Replace NaN with ""
reddit_df.loc[reddit_df["Cleaned Data"].isna(), "Cleaned Data"] = ""

#sample testing
# twitter_df = twitter_df.iloc[:5]
# reddit_df = reddit_df.iloc[:5]


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# twitter_data_texts = twitter_df["Cleaned Data"].to_list() # Features (not-tokenized yet)
reddit_data_texts = reddit_df["Cleaned Data"].to_list() # Features (not-tokenized yet)

In [ ]:
# !pip install transformers
from transformers import DistilBertTokenizer
from transformers import DistilBertForSequenceClassification
import numpy as np
from sklearn.metrics import classification_report
import torch
save_directory = "/content/drive/MyDrive/CZ4024 Information Retrieval/DistilBERT_undersampledData_Relevance"
loaded_tokenizer = DistilBertTokenizer.from_pretrained(save_directory)
loaded_model = DistilBertForSequenceClassification.from_pretrained(save_directory, from_tf=True)

All TF 2.0 model weights were used when initializing DistilBertForSequenceClassification.

All the weights of DistilBertForSequenceClassification were initialized from the TF 2.0 model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use DistilBertForSequenceClassification for predictions without further training.


In [ ]:
import time

dfs =[reddit_data_texts] #twitter_data_texts]#
# dfs = [[""], reddit_data_texts]
for n,test_texts in enumerate(dfs):
  start = time.time()
  pred_values = []
  for i,test_text in enumerate(test_texts):
    predict_input = loaded_tokenizer.encode(test_text,truncation=True,padding=True,return_tensors="pt")
    output = loaded_model(predict_input)[0]
    prediction_value = torch.argmax(output[0], dim=-1).item()
    pred_values.append(prediction_value)
    if(i%50==0):
      print(str(i)+"/"+str(len(test_texts)), end=" || ")
    if(i%300==0):
      print("")
  end = time.time()
  print("DONE")
  if False: # twitter_data
    twitter_df["Relevance"] = pred_values
    twitter_df.to_csv("/content/drive/MyDrive/CZ4024 Information Retrieval/twitter_df1(with_prediction).csv")
    twitter_df_relevant = twitter_df[twitter_df["Relevance"] == 1]
    twitter_df_relevant.to_csv("/content/drive/MyDrive/CZ4024 Information Retrieval/twitter_df_relevant1.csv")
  else: # reddit_data
    reddit_df["Relevance"] = pred_values
    reddit_df.to_csv("/content/drive/MyDrive/CZ4024 Information Retrieval/reddit_df_undersampled(with_prediction).csv")
    reddit_df_relevant = reddit_df[reddit_df["Relevance"] == 1]
    reddit_df_relevant.to_csv("/content/drive/MyDrive/CZ4024 Information Retrieval/reddit_df_undersampled_relevant.csv")
  print("Time Taken:", end-start)

0/25503 || 
50/25503 || 100/25503 || 150/25503 || 200/25503 || 250/25503 || 300/25503 || 
350/25503 || 400/25503 || 450/25503 || 500/25503 || 550/25503 || 600/25503 || 
650/25503 || 700/25503 || 750/25503 || 800/25503 || 850/25503 || 900/25503 || 
950/25503 || 1000/25503 || 1050/25503 || 1100/25503 || 1150/25503 || 1200/25503 || 
1250/25503 || 1300/25503 || 1350/25503 || 1400/25503 || 1450/25503 || 1500/25503 || 
1550/25503 || 1600/25503 || 1650/25503 || 1700/25503 || 1750/25503 || 1800/25503 || 
1850/25503 || 1900/25503 || 1950/25503 || 2000/25503 || 2050/25503 || 2100/25503 || 
2150/25503 || 2200/25503 || 2250/25503 || 2300/25503 || 2350/25503 || 2400/25503 || 
2450/25503 || 2500/25503 || 2550/25503 || 2600/25503 || 2650/25503 || 2700/25503 || 
2750/25503 || 2800/25503 || 2850/25503 || 2900/25503 || 2950/25503 || 3000/25503 || 
3050/25503 || 3100/25503 || 3150/25503 || 3200/25503 || 3250/25503 || 3300/25503 || 
3350/25503 || 3400/25503 || 3450/25503 || 3500/25503 || 3550/25503 || 360